In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0

def build_model(hp):

    model = keras.Sequential()

    model.add(layers.Flatten(input_shape=(28, 28)))

    hp_units = hp.Int('units', min_value=250, max_value=260, step=2)

    model.add(layers.Dense(units=hp_units, activation='sigmoid'))

    model.add(layers.Dense(16, activation='tanh'))

    model.add(layers.Dense(8, activation='relu'))

    model.add(layers.Dense(4, activation='selu'))

    model.add(layers.Dense(10, activation='softmax'))

    hp_learning_rate = hp.Choice('learning_rate', values=[0.1, 0.01, 0.15])

    hp_batch_size = hp.Choice('batch_size', values=[4, 8, 16])

    model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=hp_learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model
 
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=1,
    directory='keras_tuner_dir',
    project_name='mnist_tuning')

tuner.search(x_train, y_train,
             epochs=5,
             validation_data=(x_test, y_test))
 
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

model = tuner.hypermodel.build(best_hps)
 
model.fit(x_train, y_train,
          epochs=5,
          batch_size=best_hps.get('batch_size'),
          validation_data=(x_test, y_test))

eval_result = model.evaluate(x_test, y_test)

print("Test loss:", eval_result[0])

print("Test accuracy:", eval_result[1])

Trial 5 Complete [00h 00m 53s]
val_accuracy: 0.9592000246047974

Best val_accuracy So Far: 0.9613999724388123
Total elapsed time: 00h 03m 55s
Epoch 1/10
3750/3750 [==============================] - 13s 3ms/step - loss: 2.3044 - accuracy: 0.1068 - val_loss: 2.3034 - val_accuracy: 0.1032
Epoch 2/10
3750/3750 [==============================] - 14s 4ms/step - loss: 2.3039 - accuracy: 0.1056 - val_loss: 2.3044 - val_accuracy: 0.1135
Epoch 3/10
3750/3750 [==============================] - 15s 4ms/step - loss: 2.3035 - accuracy: 0.1091 - val_loss: 2.3026 - val_accuracy: 0.1135
Epoch 4/10
3750/3750 [==============================] - 10s 3ms/step - loss: 2.3035 - accuracy: 0.1078 - val_loss: 2.3031 - val_accuracy: 0.1135
Epoch 5/10
3750/3750 [==============================] - 12s 3ms/step - loss: 2.3033 - accuracy: 0.1079 - val_loss: 2.3041 - val_accuracy: 0.1032
Epoch 6/10
3750/3750 [==============================] - 14s 4ms/step - loss: 2.3035 - accuracy: 0.1079 - val_loss: 2.3043 - val_accur